In [90]:
import pandas as pd

In [91]:
da=pd.read_csv("titanic_cleaned.csv")
dd=pd.read_csv("ticket_fares.csv")

In [92]:
df = da.merge(dd, how="left", on="Ticket") 

In [93]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare_x', 'Embarked', 'familySize', 'IsAlone',
       'Fare_y'],
      dtype='object')

In [94]:
df=df.drop(columns=["Fare_x"]).rename(columns={"Fare_y":"Fare"})

In [95]:
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Embarked,familySize,IsAlone,Fare
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,S,1,0,7.2500
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,C,1,0,71.2833
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,S,0,1,7.9250


In [96]:
ageBins=[-1,12,19,59,120]
ageLabels=["Child","Teen","Adult","Senior"]
df["AgeGroup"]=pd.cut(df["Age"],bins=ageBins,labels=ageLabels)

In [97]:
survivalRates=df.groupby(["Sex","AgeGroup"])["Survived"].mean().unstack() # unstack is used to remove the last line saying the dtpye of the object
survivalRates

C:\Users\Abdullah\AppData\Local\Temp\ipykernel_11420\1587632302.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  survivalRates=df.groupby(["Sex","AgeGroup"])["Survived"].mean().unstack() # unstack is used to remove the last line saying the dtpye of the object


AgeGroup,Child,Teen,Adult,Senior
Sex,,,,
female,0.432432,0.753247,0.699561,1.000000
male,0.379562,0.086957,0.202624,0.111111


In [98]:
childSurvivalRate=df[df['AgeGroup']=='Child']['Survived'].mean()
womenSurvivalRate=df[(df["Sex"]=="female") & (df["AgeGroup"]!="Child")]["Survived"].mean()
menSurvivalRate=df[(df["Sex"]=="male") & (df["AgeGroup"]!="Child")]["Survived"].mean()

In [99]:
with open("report.txt","w") as file:
    file.write("---Hypothesis 1: Women And Childern First!---\n")
    file.write("\nSurvival Analysis by age and sex group\n")
    file.write(f"\n{survivalRates}\n")
    file.write("--Conclusion--\n")
    file.write(
        f"Childern Survival rate: {childSurvivalRate:.2%}\n"
        f"Women Survival rate: {womenSurvivalRate:.2%}\n"
        f"Men Survival rate: {menSurvivalRate:.2%}\n"
        f"Thus the data shows that the *Women And Childern First* drill was followed."
    )


In [100]:
survivalRatesbyPclass = df.groupby(['Pclass'])['Survived'].mean()
survivalRatesbyPclass

Pclass
1    0.689055
2    0.480000
3    0.239057
Name: Survived, dtype: float64

In [101]:
df["FareBin"]=pd.qcut(df["Fare"],q=4,labels=["Low","Medium","High","VeryHigh"],precision=2)
survivalRatesByFareBin=df.groupby("FareBin")["Survived"].mean()

C:\Users\Abdullah\AppData\Local\Temp\ipykernel_11420\3700694963.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  survivalRatesByFareBin=df.groupby("FareBin")["Survived"].mean()


In [102]:
highPclass=survivalRatesbyPclass[1].round(2)
lowPclass=survivalRatesbyPclass[3].round(2)

highFare=survivalRatesByFareBin["VeryHigh"].round(2)
lowFare=survivalRatesByFareBin["Low"].round(2)

In [103]:
with open("report.txt","a") as file:
    file.write("\n\n\n---\n\n")
    file.write("Hypothesis 2: Wealth\n")
    file.write("\nMethod A (Pclass) Survival Rates:\n")
    file.write(survivalRatesbyPclass.apply(lambda x: f'{x:.2%}').to_string() + "\n\n")

    file.write("Method B (FareBin) Survival Rates:\n")
    file.write(survivalRatesByFareBin.apply(lambda x: f'{x:.2%}').to_string() + "\n\n")

    file.write("Conclusion Comparison of Methods A and B:\n")
    file.write(
        f"The data clearly shows a correlation between wealth and survival\n. "
        f"Passengers in the highest wealth bracket (1st Class and Very High Fare) had advantage: \n"
        f" 1st Class passengers survived at a rate of {highPclass:.2%}, and those paying the highest fares survived at {highFare:.2%}.\n"
        f"While, passengers in the lowest wealth bracket (3rd Class and Low Fare) were the least likely to survive: \n"
        f" 3rd Class passengers had a survival rate of only {lowPclass:.2%}, while those in the lowest fare quantile survived at {lowFare:.2%}.\n"
        f"Because both Method A (discrete class) and Method B (continuous fare) show similar trends higher wealth equals higher survival the hypothesis that socio-economic status determined access to lifeboats is strongly supported."
    )